In [1]:
!pip install pybullet imageio-ffmpeg

import os
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import cv2
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 4.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import imageio_ffmpeg
import pybullet as p
import math

# Configura la simulazione e la fotocamera
p.connect(p.DIRECT)  # Usa la modalità senza GUI per evitare un impatto sulle prestazioni
p.resetSimulation() # Lui va chiamato la mattina appena apri il computer
p.setGravity(0, 0, -9.81)
p.setTimeStep(0.01)  # Imposta il passo temporale per la simulazione

# Aggiungi il percorso dei dati PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [3]:
!git clone https://github.com/Gaianeve/Real_Mega_Fufi.git
%cd Real_Mega_Fufi/Robot/1LegTrail/

Cloning into 'Real_Mega_Fufi'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 841 (delta 235), reused 25 (delta 25), pack-reused 557 (from 1)
Receiving objects: 100% (841/841), 8.24 MiB | 7.72 MiB/s, done.
Resolving deltas: 100% (548/548), done.
/content/Real_Mega_Fufi/Robot/1LegTrail


In [ ]:
plane_pos = [0,0,0]
plane = p.loadURDF("plane.urdf", plane_pos, useFixedBase=True)

# position of the base of the model
PIP_position = [0,0,0]
orientation  = [0,0,0]
PIP_orientation = p.getQuaternionFromEuler(orientation)
PIPPA_id = p.loadURDF("PIPPA_oneleg_no_joint.urdf", basePosition = PIP_position, baseOrientation = PIP_orientation, \
                      useFixedBase=True)

In [ ]:
link_name_to_index = {p.getBodyInfo(PIPPA_id)[0].decode('UTF-8'):-1,}

for id in range(p.getNumJoints(PIPPA_id)):
  name = p.getJointInfo(PIPPA_id, id)[12].decode('UTF-8')
  link_name_to_index[name] = id

In [ ]:
# Carica il giunto elastico come soft body
giunto_id = p.loadSoftBody("giunto.obj", basePosition=[0, 0, 0.225],  # adattare la posizione di base se necessario
                           mass=0.0569, useNeoHookean=1, NeoHookeanMu=76.92, NeoHookeanLambda=115.38, NeoHookeanDamping=0.01,
                           collisionMargin=0.005, useSelfCollision=0, frictionCoeff=0.5)

# Ancoraggio del giunto alla colonna e alla gamba
index_leg = link_name_to_index["Gamba"]
index_column = link_name_to_index["Colonna"]
p.createSoftBodyAnchor(giunto_id, nodeIndex= 0, bodyUniqueId= PIPPA_id, linkIndex= index_column)  # Ancora alla colonna
p.createSoftBodyAnchor(giunto_id, nodeIndex=-1, bodyUniqueId = PIPPA_id, linkIndex= index_leg)  # Ancora alla gamba


In [ ]:
# Parametri della fotocamera
cam_target_pos = [0, 0, 0]
cam_distance = 2.0
cam_yaw, cam_pitch, cam_roll = 50, -30.0, 0
cam_width, cam_height = 480, 360
cam_up_axis_idx = 2
cam_fov = 60
cam_near_plane, cam_far_plane = 0.01, 100

# Imposta la registrazione del video
vid = imageio_ffmpeg.write_frames('simulation_output.mp4', (cam_width, cam_height), fps=30)
vid.send(None)  # Inizializza con un fotogramma vuoto

# Parametri per la simulazione
simulation_time = 180
sampling_frequency = 1 / 100
num_points = int(simulation_time * sampling_frequency)

# Liste per registrare le posizioni
leg_positions_x, leg_positions_y, leg_position_z = [], [], []

# Forza esterna da applicare
force_module = np.array([50, 0, 0])
force_position = np.array([0, 0, 0.99])  # Posizione dove applicare la forza

In [ ]:
# Ciclo principale della simulazione
for step in range(1000):
    print(f'\rtimestep {step}...', end='')
    if step == 100:
      # Applicare la forza esterna al primo passo della simulazione
      p.applyExternalForce(
          objectUniqueId=PIPPA_id,
          linkIndex=link_name_to_index['Link_colonna'],
          forceObj=force_module,
          posObj=force_position,
          flags=p.WORLD_FRAME
      )

    # Registrazione della posizione della gamba
    leg_position = p.getLinkState(PIPPA_id, link_name_to_index['Gamba_Link'])[0]
    leg_positions_x.append(leg_position[0])
    leg_positions_y.append(leg_position[1])
    leg_position_z.append(leg_position[2])

    # Calcola la vista e la proiezione della fotocamera
    cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(
        cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx
    )
    cam_projection_matrix = p.computeProjectionMatrixFOV(
        cam_fov, cam_width / cam_height, cam_near_plane, cam_far_plane
    )

    # Cattura l'immagine
    image = p.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]

    # Scrivi il fotogramma nel video
    vid.send(np.ascontiguousarray(image))

    p.stepSimulation()  # Avanza di un passo nella simulazione

# Rilascia le risorse video
vid.close()

# Disconnessione dalla simulazione
p.disconnect()


In [ ]:
# Ora visualizziamo il video registrato nel notebook
mp4 = open('simulation_output.mp4', 'rb').read()  # Leggi il file video
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()  # Convertilo in Base64

# Visualizza il video nel notebook Jupyter
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)

In [ ]:
#plotting positions
plt.figure('Leg positions')
plt.plot(leg_positions_x, label='x')
plt.plot(leg_positions_y, label='y')
plt.plot(leg_position_z, label='z')
plt.legend()
plt.grid()
plt.minorticks_on()
#plt.ylim(-0.000001,0.0001)
#plt.xlim(45, 1000)
plt.show()

In [ ]:
plt.figure('xy motion')
plt.plot(leg_positions_x, leg_positions_y, label = 'leg1')
plt.legend()
plt.grid()
plt.minorticks_on()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

# Supponiamo che leg_positions_x sia un array definito
time = np.arange(len(leg_positions_x[45:-1]))  # Array di tempi in base agli indici
y_data = leg_positions_x[45:-1]  # Dati del segnale (senza valore assoluto)

# Trova i massimi locali usando find_peaks
peaks, _ = find_peaks(y_data)  # Restituisce gli indici dei picchi e altre informazioni

peak_values = []
time_peaks = []
# Estrai i valori dei massimi locali per creare l'inviluppo
for value in peaks:
    peak_values.append(y_data[value])
    time_peaks.append(time[value])

# Definisci la funzione esponenziale da adattare
def exp_decay(t, A, tau):
    return A * np.exp(-t / tau)

# Esegui il fit esponenziale sui massimi
params, covariance = curve_fit(exp_decay, time_peaks, peak_values, p0=(peak_values[0], 100))

# Estrai il parametro tau
A_fit, tau_fit = params
print(f"Stima di tau: {tau_fit}")

# Visualizza i risultati
plt.figure(figsize=(10, 6))
plt.plot(time, y_data, 'b-', label='Dati originali')
plt.plot(time[peaks], peak_values, 'go', label='Massimi locali (inviluppo)')
plt.plot(time, exp_decay(time, *params), 'r--', label=f'Fit esponenziale (tau = {tau_fit:.3f})')
plt.xlabel('Tempo')
plt.ylabel('Ampiezza')
plt.legend()
plt.title('Fit esponenziale dell\'inviluppo (massimi locali)')
plt.show()
